In [1]:
#!/usr/bin/env python3

# core openmm imports
from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *

from sys import stdout

# mdtraj provides extended trajectory output formats such as HDF5 and NetCDF
import mdtraj as mdt

# ngl view permits visualization of the system
import nglview as ng

Let us run a MD simulation of alanine dipeptide in vacuum using the AMBER 99 forcefield: first we need an input object (e.g. a pdb), and a forcefield definition (several are provided with the OpenMM installation)

In [2]:
# Input Files
print('Parsing PDB file...')
pdb = PDBFile('input/ala2.pdb')

print('Parsing FF file(s)...')
forcefield = ForceField('amber99sbildn.xml')


Parsing PDB file...
Parsing FF file(s)...


Define simulation parameters here: periodic boundaries if any, nonbonded interaction cutting scheme, rigid constraints for some of the bonds, etc.

In [3]:
# System Configuration
nonbondedMethod = CutoffNonPeriodic
nonbondedCutoff = 1.4*nanometers
constraints = HBonds
constraintTolerance = 0.00001

Create an Integrator : we use 'Langevin' at a temperature of 300 K ; for systems where a simulation box is defined it is also possible to add a barostat and then run in the NPT ensemble

In [4]:
# Integration Options
dt = 0.002*picoseconds
temperature = 300.00*kelvin
friction = 2/picosecond

integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)


An OpenMM platform object is used for providing a level of hardware abstraction : one can choose Reference, CPU, OpenCL or CUDA

In [5]:
platform = Platform.getPlatformByName('CPU')
platformProperties = {'CpuThreads':'1'}

#platform = Platform.getPlatformByName('OpenCL')
#platformProperties = {'OpenCLPrecision':'mixed'}

#platform = Platform.getPlatformByName('CUDA')
#platformProperties = {'CudaPrecision': 'mixed'}


A system object is created from the pdb, the forcefield and some of the simulation parameters above defined : it contains coordinates, velocities,
atom types, and all the data required for energy/force evaluations.

But the computation is performed within a simulation object (using the above defined platform which actually contains the harware dependent compiled code).

In [6]:
# Prepare the Simulation
print('Building system...')
topology = pdb.topology
positions = pdb.positions
system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod,
                                 nonbondedCutoff=nonbondedCutoff,
                                 constraints=constraints)

simulation = Simulation(topology, system, integrator,
                        platform, platformProperties)

# set initial positions to what was one obtained from PDB parsing
simulation.context.setPositions(positions)


Building system...


Energy minimization is performed using the L-BFGS method (https://en.wikipedia.org/wiki/Limited-memory_BFGS)

In [7]:
# Minimize energy
print('Performing energy minimization...')
simulation.minimizeEnergy()

# after minimization of the energy, set initial velocities to random distribution for a given temperature
simulation.context.setVelocitiesToTemperature(temperature)


Performing energy minimization...


Perform some steps of temperature equilibration :

In [8]:
equilibrationSteps = 25000
printFreq = 250

# reporters objects are used for terminal IO and trajectory IO
#  here we use the StateDataReporter for regularly printing simulation time, energy, etc. to a terminal
simulation.reporters.append(StateDataReporter(stdout, printFreq, step=True, 
    time=True, potentialEnergy=True, kineticEnergy=True, temperature=True))

print('Equilibrating...')
simulation.step(equilibrationSteps)

Equilibrating...
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Temperature (K)"
250,0.5000000000000003,65.97621745816829,52.80865284656352,249.07503991127038
500,1.0000000000000007,90.83804239105564,63.314967305627434,298.6286746312288
750,1.500000000000001,76.24818299879658,93.83016043810987,442.5553331933548
1000,2.0000000000000013,74.28446459708664,57.2472167117994,270.0097809487762
1250,2.4999999999999463,69.53316955153349,59.598482346929735,281.0996601701545
1500,2.999999999999891,99.53376544694227,63.73487284655278,300.60918319805074
1750,3.499999999999836,81.6188927769567,64.82392414170178,305.7457560925117
2000,3.999999999999781,87.55180378077155,60.69093107039214,286.25225723011
2250,4.499999999999726,82.93759530632212,49.45763853926312,233.26978873852818
2500,4.999999999999671,82.27019078484062,41.6705033866983,196.54131917209128
2750,5.499999999999616,74.2721741877815,57.32111229749853,270.35831371689574
3000,5.999999999999561,102.9801728834586

Now that the system is temperature equilirated we can run the simulation ; we use the mdtraj package which provides extra trajectory formats (e.g. HDF5)

In [9]:
# reste step and time counters
simulation.currentStep = 0
simulation.context.setTime(0.0)

# save the equilibration results to file : state is platform independent but less precise, checkpoint file
#  contains the binary representation but it is platform dependent
simulation.saveState('output/eq.state')
simulation.saveCheckpoint('output/eq.chk')

simsteps = 5e5 # 1 ns
svfreq=500

simulation.reporters.append(StateDataReporter(stdout, svfreq, step=True, 
    time=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True,
    temperature=True, progress=True,remainingTime=True, speed=True, totalSteps=simsteps))

hdf5rep = mdt.reporters.HDF5Reporter('output/sim_1ns.hdf5', svfreq,
                                     coordinates=True, time=True, cell=True, potentialEnergy=True,
                                     kineticEnergy=True, temperature=True, velocities=True)
simulation.reporters.append(hdf5rep)

simulation.step(simsteps)

simulation.saveState('output/sim_1ns.state')
simulation.saveCheckpoint('output/sim_1ns.chk')

hdf5rep.close()


250,0.5000000000000003,79.20145629692777,65.52308851973791,309.0434049810351
500,1.0000000000000007,79.79669556933462,60.36619601819682,284.72062573174276
#"Progress (%)","Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Time Remaining"
0.1%,500,1.0000000000000007,79.79669556933462,60.36619601819682,140.16289158753142,284.72062573174276,0,--
750,1.500000000000001,81.67750545238897,55.02792116925554,259.54234623802716
1000,2.0000000000000013,112.65683964960257,81.69491436049145,385.31874906957347
0.2%,1000,2.0000000000000013,112.65683964960257,81.69491436049145,194.35175401009403,385.31874906957347,2.07e+03,0:41
1250,2.4999999999999463,98.78453973452025,58.020761892353065,273.658251158372
1500,2.999999999999891,62.35126871301344,59.7355845458823,281.7463105548036
0.3%,1500,2.999999999999891,62.35126871301344,59.7355845458823,122.08685325889573,281.7463105548036,2.24e+03,0:38
1750,3.499999999999836,79.7

/home/fhedin/bin/miniconda3/envs/openmm-tutorials/lib/python3.5/site-packages/mdtraj/formats/hdf5.py:198: UserWarning: The .h5 extension is recommended.
  warnings.warn('The .h5 extension is recommended.')


2750,5.499999999999616,94.80953482798503,78.49600787339395,370.2309231547323
3000,5.999999999999561,74.3500686654474,64.45373258234584,303.9997263710321
0.6%,3000,5.999999999999561,74.3500686654474,64.45373258234584,138.80380124779322,303.9997263710321,2.28e+03,0:37
3250,6.499999999999506,84.25215862650809,76.36348517000553,360.17275751658485
3500,6.999999999999451,94.00494119243211,57.78430387390824,272.5429833526848
0.7%,3500,6.999999999999451,94.00494119243211,57.78430387390824,151.78924506634036,272.5429833526848,2.29e+03,0:37
3750,7.499999999999396,87.36806770818426,58.66887356884041,276.71510705199006
4000,7.999999999999341,105.40174496843068,52.41841117453828,247.23444268344215
0.8%,4000,7.999999999999341,105.40174496843068,52.41841117453828,157.82015614296895,247.23444268344215,2.29e+03,0:37
4250,8.499999999999508,92.95051734548214,69.02266045984726,325.5493367492192
4500,8.999999999999675,103.03225483490647,60.5476002265387,285.5762290845514
0.9%,4500,8.999999999999675,103.032

18000,35.99999999999803,82.49086034611923,65.36840156551435,308.3138150896033
3.6%,18000,35.99999999999803,82.49086034611923,65.36840156551435,147.85926191163358,308.3138150896033,2.22e+03,0:37
18250,36.49999999999864,96.23231377130864,72.05655974209212,339.85889671863407
18500,36.999999999999254,110.63853072298971,48.191118168251734,227.29617276929167
3.7%,18500,36.999999999999254,110.63853072298971,48.191118168251734,158.82964889124145,227.29617276929167,2.22e+03,0:37
18750,37.499999999999865,92.27138370836884,52.88482189243956,249.43429558470524
19000,38.000000000000476,67.91105792096509,84.97893278779105,400.80801033184446
3.8%,19000,38.000000000000476,67.91105792096509,84.97893278779105,152.88999070875616,400.80801033184446,2.21e+03,0:37
19250,38.50000000000109,55.35042987525452,58.968437247898706,278.12801632529045
19500,39.0000000000017,79.15125686990805,42.1235393553739,198.67808930154285
3.9%,19500,39.0000000000017,79.15125686990805,42.1235393553739,121.27479622528196,198.6780

33000,66.00000000002758,91.30942558488479,63.47596418990365,299.3880256696505
6.6%,33000,66.00000000002758,91.30942558488479,63.47596418990365,154.78538977478843,299.3880256696505,2.2e+03,0:36
33250,66.50000000002642,109.22182046162024,69.6519866643481,328.5175898868005
33500,67.00000000002525,90.1199436262808,73.14868575478496,345.00997169473635
6.7%,33500,67.00000000002525,90.1199436262808,73.14868575478496,163.26862938106575,345.00997169473635,2.2e+03,0:36
33750,67.50000000002409,90.27225883618163,69.7487350202253,328.97390905615106
34000,68.00000000002292,104.92109393970551,75.7035017544193,357.05990788462134
6.8%,34000,68.00000000002292,104.92109393970551,75.7035017544193,180.62459569412482,357.05990788462134,2.2e+03,0:36
34250,68.50000000002176,82.8315396478834,63.726475448331435,300.56957638771974
34500,69.00000000002059,86.1989973478064,55.42946113660171,261.4362325237331
6.9%,34500,69.00000000002059,86.1989973478064,55.42946113660171,141.6284584844081,261.4362325237331,2.21e+0

...

86.8%,434000,867.9999999932983,73.28997059982109,62.417574430872136,135.7075450306932,294.3960696688337,2.14e+03,0:05
434250,868.4999999932865,110.07795424171516,36.4265488049461,171.80790662816923
434500,868.9999999932746,83.68544781952863,40.61720118138335,191.5733589101676
86.9%,434500,868.9999999932746,83.68544781952863,40.61720118138335,124.30264900091198,191.5733589101676,2.14e+03,0:05
434750,869.4999999932628,77.55744728048963,71.04548152768395,335.0900883315154
435000,869.999999993251,78.65052971861678,62.660552897817894,295.542091543846
87.0%,435000,869.999999993251,78.65052971861678,62.660552897817894,141.31108261643467,295.542091543846,2.14e+03,0:05
435250,870.4999999932392,77.79788275081174,67.95764450628117,320.5261279218447
435500,870.9999999932273,84.14544325532242,77.1551792215848,363.90682791727403
87.1%,435500,870.9999999932273,84.14544325532242,77.1551792215848,161.30062247690722,363.90682791727403,2.14e+03,0:05
435750,871.4999999932155,98.36552855665838,51.349397912

449000,897.9999999925889,87.91515919082126,73.41065731749542,346.2455755952432
89.8%,449000,897.9999999925889,87.91515919082126,73.41065731749542,161.32581650831668,346.2455755952432,2.13e+03,0:04
449250,898.499999992577,104.64088826995994,80.83316709112424,381.25426864939107
449500,898.9999999925652,87.45344509608805,58.785651144936324,277.2658951872478
89.9%,449500,898.9999999925652,87.45344509608805,58.785651144936324,146.23909624102436,277.2658951872478,2.13e+03,0:04
449750,899.4999999925534,115.7022855873802,54.56476205671472,257.35782971993757
450000,899.9999999925416,81.95998333933571,49.42060303486326,233.09510865791086
90.0%,450000,899.9999999925416,81.95998333933571,49.42060303486326,131.38058637419897,233.09510865791086,2.13e+03,0:04
450250,900.4999999925298,90.93312465198646,43.50475623922713,205.19267795184302
450500,900.9999999925179,95.91369488438608,60.387314543727314,284.82023246862843
90.1%,450500,900.9999999925179,95.91369488438608,60.387314543727314,156.301009428113

463750,927.4999999918913,87.2196583138645,73.37863023886406,346.09451804201376
464000,927.9999999918795,123.18672075864859,69.24366293048269,326.59170757723376
92.8%,464000,927.9999999918795,123.18672075864859,69.24366293048269,192.43038368913128,326.59170757723376,2.13e+03,0:02
464250,928.4999999918676,80.57828677739855,70.89931780894052,334.40069876906296
464500,928.9999999918558,80.06043904078673,60.86529844195003,287.0746709386398
92.9%,464500,928.9999999918558,80.06043904078673,60.86529844195003,140.92573748273676,287.0746709386398,2.13e+03,0:02
464750,929.499999991844,79.62375364679514,76.3324801379747,360.02652050476473
465000,929.9999999918322,79.06811008903586,90.34132955311654,426.1000622276623
93.0%,465000,929.9999999918322,79.06811008903586,90.34132955311654,169.4094396421524,426.1000622276623,2.13e+03,0:02
465250,930.4999999918203,84.27669659784604,69.92976616997471,329.8277528567121
465500,930.9999999918085,80.80288044661901,85.62773466504242,403.8681215969515
93.1%,46550

95.7%,478500,956.9999999911937,98.08577718530141,73.95446269289926,172.04023987820068,348.81046483747065,2.13e+03,0:01
478750,957.4999999911819,91.00278260646681,65.18685984708297,307.45756316257086
479000,957.9999999911701,81.09045845098768,63.45163123901734,299.27325791121564
95.8%,479000,957.9999999911701,81.09045845098768,63.45163123901734,144.542089690005,299.27325791121564,2.13e+03,0:01
479250,958.4999999911582,78.18479520877789,91.87181078292916,433.31866472651086
479500,958.9999999911464,106.80615243699913,69.7509141460354,328.9841870280148
95.9%,479500,958.9999999911464,106.80615243699913,69.7509141460354,176.55706658303453,328.9841870280148,2.13e+03,0:01
479750,959.4999999911346,78.52239499263419,72.58760212606805,342.36358857978973
480000,959.9999999911228,59.046634134887555,52.37780758550998,247.04293352703272
96.0%,480000,959.9999999911228,59.046634134887555,52.37780758550998,111.42444172039754,247.04293352703272,2.13e+03,0:01
480250,960.4999999911109,87.18403169122114,49.

493500,986.9999999904843,85.5119763712742,65.46754202216933,308.7814167394675
98.7%,493500,986.9999999904843,85.5119763712742,65.46754202216933,150.97951839344353,308.7814167394675,2.12e+03,0:00
493750,987.4999999904725,94.55772200124608,60.59114350544917,285.7816034568915
494000,987.9999999904607,88.63882120699208,45.26839587000485,213.51098542129108
98.8%,494000,987.9999999904607,88.63882120699208,45.26839587000485,133.90721707699691,213.51098542129108,2.12e+03,0:00
494250,988.4999999904488,105.61161749213866,37.985256672714065,179.15964168383033
494500,988.999999990437,89.09123052804262,69.73486696543281,328.90849958034096
98.9%,494500,988.999999990437,89.09123052804262,69.73486696543281,158.82609749347543,328.90849958034096,2.12e+03,0:00
494750,989.4999999904252,81.55008288342722,69.80710877728417,329.2492322292578
495000,989.9999999904134,72.10112585288529,93.95903256698891,443.1631659804789
99.0%,495000,989.9999999904134,72.10112585288529,93.95903256698891,166.0601584198742,443.1

When simulation finished we can have a look at the trajectory using an external trajectory viewer (PyMOL, VMD,...) or using nglview within this jupyter notebook:

In [2]:
traj = mdt.load_hdf5("output/sim_1ns.hdf5")

view = ng.show_mdtraj(traj)
view.add_ball_and_stick("all")

view

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
